In [1]:
from code.io.EventProcessor import EventProcessor
from code.io.TrackingProcessor import TrackingProcessor
from code.io.PossessionProcessor import PossessionProcessor
from code.io.ActionProcessor import ActionProcessor

from code.util.StatsUtil import StatsUtil
from code.util.ShotRegionUtil import ShotRegionUtil
from code.util.VisUtil import VisUtil
from code.util.FeatureUtil import FeatureUtil

import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['animation.ffmpeg_path'] = r'C:\FFmpeg\bin\ffmpeg.exe'

GAME_ID = "0022300869"

event_df = EventProcessor.load_games()
tracking_df = TrackingProcessor.load_games()
possession_df = PossessionProcessor.load_games()

In [2]:
off_rebounds_df = EventProcessor.extract_off_rebounds(event_df)
oreb_ppp = FeatureUtil.calculate_oreb_ppp(event_df, off_rebounds_df)
oreb_ppp

1.1025641025641026